In [1]:
import os 

os.environ["ISISROOT"] = "/usgs/cpkgs/anaconda3_linux/envs/isis3.7.0"

import pysis
from pysis import isis

import knoten
from knoten import vis

import json

from ale import util

import ale
from ale.drivers.mro_drivers import MroCtxPds3LabelNaifSpiceDriver
from ale.formatters.usgscsm_formatter import to_usgscsm

from IPython.display import Image

import plotly
plotly.offline.init_notebook_mode(connected=True)

ImportError: cannot import name 'vis'

In [ ]:
url = 'https://pds-imaging.jpl.nasa.gov/data/mro/mars_reconnaissance_orbiter/ctx/mrox_3365/data/K11_057871_1602_XN_19S163W.IMG'

# !wget -N -P data/ {url}
fileName = 'data/' + os.path.split(url)[1]
camera = knoten.csm.create_csm(fileName)

In [ ]:
# Set the output location of the resulting .cub
cub_loc = os.path.splitext(fileName)[0] + '.cub'

try: 
    isis.mroctx2isis(from_=fileName, to=cub_loc)
except ProcessError as e:
    print(e.stderr)

try:
    results = isis.spiceinit(from_=cub_loc, shape='ellipsoid')
except ProcessError as e:
    print(e.stderr)

try:
    results = isis.footprintinit(from_=cub_loc)
except ProcessError as e:
    print(e.stderr)
    
util.write_metakernel_from_cube(cub_loc, '/home/krodriguez/mro_2018_v05.tm')

with MroCtxPds3LabelNaifSpiceDriver(fileName) as driver:
    usgscsmString = to_usgscsm(driver)
    usgscsm_dict = json.loads(usgscsmString)
    
    csm_isd = os.path.splitext(fileName)[0] + '.json'
    json.dump(usgscsm_dict, open(csm_isd, 'w'))

In [ ]:
csmisis_diff_lv_plot, csmisis_diff_ephem_plot, external_orientation_data = vis.external_orientation_diff(csm_isd, cub_loc, 10, 50, 600, 600)

In [ ]:
csmisis_diff_lv_plot_bytes = csmisis_diff_lv_plot.to_image(format="png")
csmisis_diff_ephem_plot_bytes = csmisis_diff_ephem_plot.to_image(format="png")
Image(csmisis_diff_lv_plot_bytes)

In [ ]:
Image(csmisis_diff_ephem_plot_bytes)

In [ ]:
external_orientation_data[['diffx', 'diffy', 'diffz', 'diffu', 'diffv', 'diffw']].describe()

In [ ]:
isis2csm_plot, csm2isis_plot, isiscsm_plotlatlon, isiscsm_plotbf, isis2csm_data, csm2isis_data, isiscsm_latlondata, isiscsm_bfdata = vis.reprojection_diff(csm_isd, cub_loc, 10, 50, 500, 500)

In [ ]:
Image(isis2csm_plot.to_image())

In [ ]:
isis2csm_data[['diff line', 'diff sample']].describe()

In [ ]:
Image(csm2isis_plot.to_image())

In [ ]:
csm2isis_data[['diff line', 'diff sample']].describe()

In [ ]:
Image(isiscsm_plotlatlon.to_image())

In [ ]:
Image(isiscsm_plotbf.to_image())

In [ ]:
isiscsm_bfdata[['diffx', 'diffy', 'diffz']].describe()